# Task03:  Implement PIT Join in Python

## Problem:
Let suppose I am building a recommendation system for an e-commerce platform. I want to recommend products to users based on their browsing history and previous purchases. However, I want to ensure that the recommendations are relevant to the time when the user interacted with the platform.

## Goal :  The goal is to match the interaction data with the most recent relevant feature record for the same product at or before the interaction time.

#### Scenario: Product Recommendation Based on Browsing History

1. Base Data (User Interaction Data): This table contains records of user interactions (like clicks, views, or purchases) on the platform.

2. Feature Data (Product Popularity Data): This table contains data about the popularity of products (e.g., number of purchases) over time.

Objective: For each user interaction, join the product with popularity data available up to that PIT while considering a TTL to ensure that only recent popularity data is used.

#### Step 1: Setting Up the Data

1. User Interaction Data (Base Data):

  --> Contains records of user interactions with the platform.

  --> Columns: Interaction_id, UserId, ProductId, InteractionType,
  InteractionTime.

2. Product Popularity Data (Feature Data):

  --> Contains records of product popularity over time.

  --> Columns: ProductId, PopularityScore, RecordTime.

### Step 1: Setup the Environment

Start by importing the necessary libraries and creating example dataframes.

In [46]:
import pandas as pd
from datetime import datetime, timedelta

In [48]:
# Base data (User Interaction Data)
base_data = {
    'Interaction_id': ['1', '2', '3', '4', '5'],
    'UserId': ['101', '102', '103', '101', '103'],
    'ProductId': ['P1', 'P2', 'P3', 'P1', 'P3'],
    'InteractionType': ['view', 'purchase', 'click', 'purchase', 'view'],
    'InteractionTime': [datetime.strptime("2023-08-01 12:00:00", "%Y-%m-%d %H:%M:%S"),datetime.strptime("2023-08-01 15:00:00", "%Y-%m-%d %H:%M:%S"),
        datetime.strptime("2023-08-02 18:00:00", "%Y-%m-%d %H:%M:%S"),datetime.strptime("2023-08-03 11:00:00", "%Y-%m-%d %H:%M:%S"),
        datetime.strptime("2023-08-03 13:00:00", "%Y-%m-%d %H:%M:%S")
    ]
}

base_df = pd.DataFrame(base_data)

# Feature data (Product Popularity Data)
feature_data = {
  'ProductId': ['P1', 'P2', 'P3', 'P1', 'P2', 'P3', 'P1', 'P2', 'P3'],
  'PopularityScore': [0.5, 0.3, 0.1, 0.7, 0.4, 0.2, 0.8, 0.5,0.3],
  'RecordTime': [datetime.strptime("2023-08-01 10:00:00", "%Y-%m-%d %H:%M:%S"),datetime.strptime("2023-08-01 11:00:00", "%Y-%m-%d %H:%M:%S"),
    datetime.strptime("2023-08-01 14:00:00", "%Y-%m-%d %H:%M:%S"),datetime.strptime("2023-08-02 16:00:00", "%Y-%m-%d %H:%M:%S"),
    datetime.strptime("2023-08-02 15:00:00", "%Y-%m-%d %H:%M:%S"), datetime.strptime("2023-08-02 20:00:00", "%Y-%m-%d %H:%M:%S"),
    datetime.strptime("2023-08-03 09:00:00", "%Y-%m-%d %H:%M:%S"), datetime.strptime("2023-08-03 10:00:00", "%Y-%m-%d %H:%M:%S"),
    datetime.strptime("2023-08-03 12:00:00", "%Y-%m-%d %H:%M:%S")
    ]
}

feature_df = pd.DataFrame(feature_data)

In [49]:
base_df

,Interaction_id,UserId,ProductId,InteractionType,InteractionTime
0,1,101,P1,view,2023-08-01 12:00:00
1,2,102,P2,purchase,2023-08-01 15:00:00
2,3,103,P3,click,2023-08-02 18:00:00
3,4,101,P1,purchase,2023-08-03 11:00:00
4,5,103,P3,view,2023-08-03 13:00:00


In [50]:
feature_df

,ProductId,PopularityScore,RecordTime
0,P1,0.5,2023-08-01 10:00:00
1,P2,0.3,2023-08-01 11:00:00
2,P3,0.1,2023-08-01 14:00:00
3,P1,0.7,2023-08-02 16:00:00
4,P2,0.4,2023-08-02 15:00:00
5,P3,0.2,2023-08-02 20:00:00
6,P1,0.8,2023-08-03 09:00:00
7,P2,0.5,2023-08-03 10:00:00
8,P3,0.3,2023-08-03 12:00:00


### Step 2: TTL setup and PIT join:
Only consider product popularity Data within the last 24 hours before the interaction.

In [51]:
ttl = timedelta(hours=24)

In [52]:
# PIT join with TTL
intermediate_df = pd.merge(base_df, feature_df, left_on='ProductId', right_on='ProductId', how='left')
intermediate_df = intermediate_df[
(intermediate_df['InteractionTime'] >= intermediate_df['RecordTime']) &
(intermediate_df['InteractionTime'] - intermediate_df['RecordTime'] <= ttl)
]

### Step 3: Selecting recent records:

In [53]:
# lag (difference in hours)
intermediate_df['lag'] = (intermediate_df['InteractionTime'] - intermediate_df['RecordTime']).dt.total_seconds() / 3600

In [54]:
# Minimum lag (for each Interaction_id and ProductId)
min_lag_df = intermediate_df.groupby(['Interaction_id', 'ProductId'])['lag'].min().reset_index()

In [55]:
#Merge back to get the corressponding popularity score with minimum lag
final_df = pd.merge(intermediate_df, min_lag_df, on=['Interaction_id', 'ProductId', 'lag'], how='inner')

In [56]:
final_df

,Interaction_id,UserId,ProductId,InteractionType,InteractionTime,PopularityScore,RecordTime,lag
0,1,101,P1,view,2023-08-01 12:00:00,0.5,2023-08-01 10:00:00,2.0
1,2,102,P2,purchase,2023-08-01 15:00:00,0.3,2023-08-01 11:00:00,4.0
2,4,101,P1,purchase,2023-08-03 11:00:00,0.8,2023-08-03 09:00:00,2.0
3,5,103,P3,view,2023-08-03 13:00:00,0.3,2023-08-03 12:00:00,1.0


### Step 3: Output


In [57]:
base_df

,Interaction_id,UserId,ProductId,InteractionType,InteractionTime
0,1,101,P1,view,2023-08-01 12:00:00
1,2,102,P2,purchase,2023-08-01 15:00:00
2,3,103,P3,click,2023-08-02 18:00:00
3,4,101,P1,purchase,2023-08-03 11:00:00
4,5,103,P3,view,2023-08-03 13:00:00


In [58]:
feature_df

,ProductId,PopularityScore,RecordTime
0,P1,0.5,2023-08-01 10:00:00
1,P2,0.3,2023-08-01 11:00:00
2,P3,0.1,2023-08-01 14:00:00
3,P1,0.7,2023-08-02 16:00:00
4,P2,0.4,2023-08-02 15:00:00
5,P3,0.2,2023-08-02 20:00:00
6,P1,0.8,2023-08-03 09:00:00
7,P2,0.5,2023-08-03 10:00:00
8,P3,0.3,2023-08-03 12:00:00


In [59]:
# Relevannt columns
final_df = final_df[['Interaction_id', 'UserId', 'ProductId', 'InteractionType', 'InteractionTime', 'PopularityScore']]

final_df

,Interaction_id,UserId,ProductId,InteractionType,InteractionTime,PopularityScore
0,1,101,P1,view,2023-08-01 12:00:00,0.5
1,2,102,P2,purchase,2023-08-01 15:00:00,0.3
2,4,101,P1,purchase,2023-08-03 11:00:00,0.8
3,5,103,P3,view,2023-08-03 13:00:00,0.3


#### How row is joined?

For example take interaction_id =1

Base Data:
  
  ->UserId = 101, ProductId = P1

  ->InteractionType = view, InteractionTime = 2023-08-01 12:00:00

Feature Data:

  ->P1 has a PopularityScore of 0.5 recorded at 2023-08-01 10:00:00.

#### Reason for Join:

-->This is the most recent record within the TTL (24 hours before the interaction).
The interaction occurred 2 hours after the popularity record, so the TTL condition is satisfied.

Result: PopularityScore = 0.5


#### Why interaction ID=3  is not not ?

The previous record with RecordTime = 2023-08-01 14:00:00 is the closest valid record that is before the interaction time.